In [34]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import glob

import gc
import xgboost as xgb
from tqdm import tqdm
import shutil
import copy
from riiid_feature_maker import *
import lightgbm as lgb

import warnings 
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
target = "answered_correctly"
features = [
 #'timestamp',
 'user_id',
 'content_id',
 'content_type_id',
 'task_container_id',
 #'user_answer',
 #'answered_correctly',
 'prior_question_elapsed_time',
 'prior_question_had_explanation',
 #'answered_correctly',
 'answered_correctly_users_mean',
 'answered_correctly_users_sum',
 'answered_correctly_users_count',
 'answered_correctly_contents_mean',
 'answered_correctly_contents_sum',
 'answered_correctly_contents_count',
 'part'
]

In [32]:
model_path = os.path.join("..","..","input","riiid-test-answer-prediction","riiid-model-lightgbm","featureV2-LightGBM_trained_50000000samples.mdl")
model = lgb.Booster(model_file=model_path)

In [26]:
SAMPLE_NUM = 10000000

In [27]:
base_dir = os.path.join("..","..")
train_df = pd.read_csv(os.path.join(base_dir,"input","riiid-test-answer-prediction","train.csv"),
                       usecols=["row_id","user_id","content_id","answered_correctly"],
                       nrows=SAMPLE_NUM)

train_df = train_df[train_df[target].notna()]
train_df = train_df[train_df[target] != -1]

content_dict = init_content_dict(train_df)
user_dict = init_user_dict(train_df)

del train_df
gc.collect()

48

In [28]:
questions = pd.read_csv(os.path.join(base_dir,"input","riiid-test-answer-prediction","questions.csv"))

In [36]:
import pickle
test_path = os.path.join("..","..","input","riiid-test-answer-prediction","iter_test.pickle")
with open(test_path,"rb") as f:
    iter_test = pickle.load(f)
    
    
import time

start = time.time()

test_size = 0

i = 0
for test_df in (iter_test):
    
    test_df['user_answer'] = np.nan
    test_df['answered_correctly'] = np.nan
    
    time_list = []
    
    a = time.time()
    if i != 0:
        str = test_df["prior_group_answers_correct"].values[0]
        str = str.replace("[","")
        str = str.replace("]","")
        str = str.split(",")
        str = np.array(str).astype("int")
        prior_test_df["answered_correctly"] = str
        #print(prior_test_df)
        train_df = prior_test_df[["user_id","content_id","answered_correctly"]]
        user_dict = update_user_dict(train_df, user_dict[0], user_dict[1])
        content_dict = update_content_dict(train_df, content_dict[0], content_dict[1])
    print("finished:",time.time()-a)
    
    a = time.time()
    _, test_df = make_base_features(test_df)
    print("finished:",time.time()-a)
    
    a = time.time()
    _ ,test_df = add_UserFeatures(test_df,user_dict)
    print("finished:",time.time()-a)
    
    a = time.time()
    _ ,test_df = add_ContentFeatures(test_df,content_dict)
    print("finished:",time.time()-a)
    
    a = time.time()
    _ ,test_df = add_QuestionFeatures(test_df,questions)
    print("finished:",time.time()-a,"\n")
    
    print(test_df.columns)
    test_df = test_df.fillna(test_df.mean())
    pred = model.predict(test_df[features])
    
    prior_test_df = copy.copy(test_df)
    i += 1
    test_size += len(test_df)

end = time.time()

elapsed_time = end - start
    
print("completed:",elapsed_time,"[sec]")
print("time per test-size:",elapsed_time/test_size)
print("estimated for 2.5million test:",(elapsed_time/test_size)*2500000/3600,"hours")

finished: 0.0
finished: 0.007978439331054688
finished: 0.1556546688079834
finished: 0.03191566467285156
finished: 0.005985736846923828 

Index(['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'prior_question_elapsed_time',
       'prior_question_had_explanation', 'answered_correctly',
       'answered_correctly_users_sum', 'answered_correctly_users_count',
       'answered_correctly_users_mean', 'answered_correctly_contents_sum',
       'answered_correctly_contents_count', 'answered_correctly_contents_mean',
       'part'],
      dtype='object')
finished: 0.0109710693359375
finished: 0.010971307754516602
finished: 0.1765275001525879
finished: 0.029919862747192383
finished: 0.005983829498291016 

Index(['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'prior_question_elapsed_time',
       'prior_question_had_explanation', 'answered_correctly',
       'answered_corre